## spark partitioning: 
    partitionBy(col) - very much dependent on the selectivity of the values (small vs large files). partition pruning possible!
        e.g. by load_id, or by date, but date (year, month is only supported in pyspark 4.0.0)
    repartition(partnum, col) - split the records into  <partnum> files using HASH algorythm. parallel processing ok, but no partition pruning
    repartitionByRange(partnum, col) - split the records into  <partnum> files using RANGE algorythm, based on data sampling. parallel processing ok, but no partition pruning
    write.option("maxRecordsPerFile", recnum) - split the records into recnum per file roughly(!).  parallel processing ok, but no partition pruning
    
    
## partition to_date datetype historical column into year/months by using spark 4.0.0 function?
    partitionedBy(pyspark.sql.functions.partitioning.months(col)) - the only reasonable partitioning would be by date (e.g. month or year) which will only be supported in pyspark 4.0.0
    

In [1]:
# setup spark on linux
from pyspark.sql import *

if SparkSession.getActiveSession() == None:
    print("Active spark session not found")
else:
    print("Active spark session found")

spark = (SparkSession.builder
         .master("local")
         .appName("SparkSession#1")
         .enableHiveSupport() # enableHiveSupport() needed to make data persistent... 
         #.config("spark.driver.allowMultipleContexts", "true")
         #.config("spark.sql.cbo.enabled", "true")
         #.config("spark.sql.cbo.optimizer", "true")
         .getOrCreate())

print('spark version:', spark.version)

spark.sql("Show databases").show(truncate=False)
spark.sql("select current_database()").show(truncate=False)
spark.sql("use mytestdb")
spark.sql("Show tables").show(truncate=False)
spark.sql("Show views").show(truncate=False)

print('Done.')

Active spark session not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 13:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark version: 3.5.5


25/04/08 13:40:28 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/08 13:40:28 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/04/08 13:40:30 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/04/08 13:40:30 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jovyan@172.17.0.2


+---------+
|namespace|
+---------+
|default  |
|mytestdb |
+---------+

+------------------+
|current_database()|
+------------------+
|default           |
+------------------+



25/04/08 13:40:32 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------------------+-----------+
|namespace|tableName            |isTemporary|
+---------+---------------------+-----------+
|mytestdb |t_my_date_partitioned|false      |
|mytestdb |v_my_date_partitioned|false      |
+---------+---------------------+-----------+

+---------+---------------------+-----------+
|namespace|viewName             |isTemporary|
+---------+---------------------+-----------+
|mytestdb |v_my_date_partitioned|false      |
+---------+---------------------+-----------+

Done.


In [14]:
# partitioning (hash, range, round-robin) test
# CANNOT FIND A WAY to PARTITION a date (to_date) into date ranges!!!

from datetime import datetime, timedelta
import random
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType
import shutil

# creating large DF:
print("Creating list: ")
ct = datetime.now()
l = []
rows = 100
for c in range(rows):
    #from_date = datetime.now() + timedelta(days = random.randint(0, 1000))
    d = datetime.now() + timedelta(days = random.randint(0, 1000))
    from_date = int((datetime.now() + timedelta(days = random.randint(0, 1000))).strftime("%Y%m%d"))
    
    #to_date = from_date + timedelta(days = random.randint(1, 100))
    to_date = int((d + timedelta(days = random.randint(1, 100))).strftime("%Y%m%d"))
    
    l.append([c, 
              ('name-' + str(c)),  
              None if random.random() < 0.1 else random.randint(1, 100),  
              None if random.random() < 0.1 else random.randint(1000, 1004),
              from_date,
              to_date,
              to_date // 100, #int(to_date.strftime("%Y%m")),
             ])

df_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("district", IntegerType(), True),
    StructField("from_date", IntegerType(), True), # StructField("from_date", DateType(), True),
    StructField("to_date", IntegerType(), True), #StructField("to_date", DateType(), True),
    # tech columns
    StructField("to_date_year_month", IntegerType(), True),
])
print(f"Done in {datetime.now() - ct}")

print("creating DF from list: ")
ct = datetime.now()
df = spark.createDataFrame(l, df_schema)
df.printSchema()
df.show(5)
print(f"Done in {datetime.now() - ct}")

spark.sql("create database if not exists mytestdb")
spark.sql("Show databases").show()
spark.sql("use mytestdb")
spark.sql("select current_database()").show()
spark.sql("Show tables").show(truncate=False)

print("Dropping table...")
spark.sql('drop table if exists t_my_date_partitioned')

# save as managed 
print("Saving table t_my_date_partitioned...")
(
    df.write
    .option("header", True)
    .mode("overwrite")
    .partitionBy("to_date_year_month") 
    .format("parquet") # parquet, csv for testing / readability
    .saveAsTable("t_my_date_partitioned")
)

# creating view to manage technical column
print("creating view v_my_date_partitioned to manage technical column DOES NOT HELP!!!...")
spark.sql("""
create or replace view v_my_date_partitioned as
SELECT *
FROM t_my_date_partitioned t
WHERE 1 = 1
--AND t.to_date_year_month = to_number(date_format(to_date, 'yyyyMM'),'000000')
AND t.to_date_year_month = floor(t.to_date / 100)
""")

print("Done.")

Creating list: 
Done in 0:00:00.001444
creating DF from list: 
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- district: integer (nullable = true)
 |-- from_date: integer (nullable = true)
 |-- to_date: integer (nullable = true)
 |-- to_date_year_month: integer (nullable = true)

+---+------+---+--------+---------+--------+------------------+
| id|  name|age|district|from_date| to_date|to_date_year_month|
+---+------+---+--------+---------+--------+------------------+
|  0|name-0| 75|    1001| 20260823|20270604|            202706|
|  1|name-1| 77|    1002| 20261005|20271113|            202711|
|  2|name-2| 67|    1004| 20261125|20261221|            202612|
|  3|name-3| 10|    1002| 20271012|20270619|            202706|
|  4|name-4| 81|    1000| 20260701|20270128|            202701|
+---+------+---+--------+---------+--------+------------------+
only showing top 5 rows

Done in 0:00:00.157459
+---------+
|namespace|
+-

In [11]:
# Querying table using SQL
print("Querying the table...")
spark.sql("""
select from_date, to_date, count(1) cnt
from t_my_date_partitioned
group by from_date, to_date
order by from_date, to_date nulls first
""").show(5, truncate=False)

# Drop table
#print("Dropping table...")
#spark.sql('drop table if exists t_my_date_partitioned')

spark.sql("Show tables").show(truncate=False)

Querying the table...
+----------+----------+---+
|from_date |to_date   |cnt|
+----------+----------+---+
|2025-04-06|2025-04-22|1  |
|2025-04-18|2025-06-21|1  |
|2025-05-19|2025-07-01|1  |
|2025-05-19|2025-08-15|1  |
|2025-06-17|2025-09-08|1  |
+----------+----------+---+
only showing top 5 rows

+---------+---------------------+-----------+
|namespace|tableName            |isTemporary|
+---------+---------------------+-----------+
|mytestdb |t_my_date_partitioned|false      |
+---------+---------------------+-----------+



In [21]:
# show SQL explain plan

def print_dict(d: dict, level: int = 0) -> None:
    """ print optionally hiearchic dict structure nicely formatted
    """

    sp = "".ljust(level * 3)
    for k in d.keys():
        print(f"{sp}{k}: {d[k]}")
        
        if type(d[k]) is dict:
            print_dict(d[k], level + 1)
            
    if level == 0:
        print()

print("exec plan for table select with partition filter:")
plan = spark.sql("""
explain
select *
FROM t_my_date_partitioned t
WHERE 1 = 1
AND t.to_date_year_month >= to_number('202710','000000')
""")

for row in plan.collect():
    None
    print_dict(row.asDict())

#####################################################################
print("exec plan for table select with partition filter and extra filter:")
plan = spark.sql("""
explain
select t.id, t.age
FROM t_my_date_partitioned t
WHERE 1 = 1
AND to_date_year_month >= to_number('202710','000000')
AND t.from_date <= to_date('20271015','yyyyMMdd')
AND t.to_date > to_date('20271015','yyyyMMdd')
""")

for row in plan.collect():
    print_dict(row.asDict())

#####################################################################
# date type for to_date 
print("exec plan for VIEW select with partition filter and extra filter using view.  DOES NOT HELP!!!...:")
plan = spark.sql("""
explain
select *
FROM v_my_date_partitioned t
WHERE 1 = 1
--AND t.to_date_year_month >= to_number('202710','000000')
AND t.from_date <= to_date('20271015','yyyyMMdd')
AND t.to_date > to_date('20271015','yyyyMMdd')
""")

# int type for to_date
plan = spark.sql("""
explain
select *
FROM v_my_date_partitioned t
WHERE 1 = 1
--AND t.to_date_year_month >= 202710
AND t.from_date <= 20271015
AND t.to_date > 20271015
""")

for row in plan.collect():
    None
    print_dict(row.asDict())

exec plan for table select with partition filter:
plan: == Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet spark_catalog.mytestdb.t_my_date_partitioned[id#361,name#362,age#363,district#364,from_date#365,to_date#366,to_date_year_month#367] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(6 paths)[file:/home/jovyan/work/various_tests/spark/spark-warehouse/mytestdb.db..., PartitionFilters: [isnotnull(to_date_year_month#367), (to_date_year_month#367 >= 202710)], PushedFilters: [], ReadSchema: struct<id:int,name:string,age:int,district:int,from_date:int,to_date:int>



exec plan for table select with partition filter and extra filter:
plan: == Physical Plan ==
org.apache.spark.sql.AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(from_date <= to_date(20271015, yyyyMMdd))" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("INT" and "DATE").; line 7 pos 4;
'Project ['t.id, 't.ag

In [57]:
# show python DF explain plan

def print_dict(d: dict, level: int = 0) -> None:
    """ print optionally hiearchic dict structure nicely formatted
    """

    sp = "".ljust(level * 3)
    for k in d.keys():
        print(f"{sp}{k}: {d[k]}")
        
        if type(d[k]) is dict:
            print_dict(d[k], level + 1)
            
    if level == 0:
        print()

print("Reading DF from table...")
spark.sql("use mytestdb")
df = (
    spark
    .read
    .table("t_my_date_partitioned")
)
df.show(4, truncate=False)

print("plan with partition filter (PartitionFilters[...]):")
df1 = (
    df
    .filter("to_date_year_month >= 202710")
    .select(df.id, df.age, df.from_date, df.to_date, df.to_date_year_month)
    .orderBy(df.from_date, df.to_date, df.id)
    )

df1.explain()

df1.show(100, truncate=False)

for row in df1.collect():
    None
    #print_dict(row.asDict())

###################################################################
print("plan with partition filter (PartitionFilters[...]) and extra filter:")
df1 = (
    df
    .filter("to_date_year_month >= 202710")
    .filter("from_date <= to_date('20271015','yyyyMMdd') AND to_date > to_date('20271015','yyyyMMdd')")
    .select(df.id, df.age, df.from_date, df.to_date, df.to_date_year_month)
    .orderBy(df.from_date, df.to_date, df.id)
    )

df1.explain()

df1.show(100, truncate=False)

for row in df1.collect():
    None
    #print_dict(row.asDict())

Reading DF from table...
+---+-------+---+--------+----------+----------+------------------+
|id |name   |age|district|from_date |to_date   |to_date_year_month|
+---+-------+---+--------+----------+----------+------------------+
|0  |name-0 |87 |1002    |2025-12-01|2026-02-02|202602            |
|9  |name-9 |65 |1003    |2025-12-19|2026-02-08|202602            |
|47 |name-47|49 |1003    |2026-01-27|2026-02-04|202602            |
|50 |name-50|62 |1002    |2026-01-26|2026-02-25|202602            |
+---+-------+---+--------+----------+----------+------------------+
only showing top 4 rows

plan with partition filter (PartitionFilters[...]):
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [from_date#513 ASC NULLS FIRST, to_date#514 ASC NULLS FIRST, id#509 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(from_date#513 ASC NULLS FIRST, to_date#514 ASC NULLS FIRST, id#509 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=2040]
      +- FileScan parquet spark_catal

In [59]:
print("plan with partition filter (PartitionFilters[...]):")
df1 = (
    df
    .filter("to_date_year_month >= 202710")
    .filter("from_date <= to_date('20271015','yyyyMMdd') AND to_date > to_date('20271015','yyyyMMdd')")
    .select(df.id, df.age, df.from_date, df.to_date, df.to_date_year_month)
    .orderBy(df.from_date, df.to_date, df.id)
    )

df1.explain()

#df1.show(100, truncate=False)

plan with partition filter (PartitionFilters[...]):
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [from_date#513 ASC NULLS FIRST, to_date#514 ASC NULLS FIRST, id#509 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(from_date#513 ASC NULLS FIRST, to_date#514 ASC NULLS FIRST, id#509 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=2175]
      +- Filter (((isnotnull(from_date#513) AND isnotnull(to_date#514)) AND (from_date#513 <= 2027-10-15)) AND (to_date#514 > 2027-10-15))
         +- FileScan parquet spark_catalog.mytestdb.t_my_date_partitioned[id#509,age#511,from_date#513,to_date#514,to_date_year_month#515] Batched: true, DataFilters: [isnotnull(from_date#513), isnotnull(to_date#514), (from_date#513 <= 2027-10-15), (to_date#514 > ..., Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/various_tests/spark/spark-warehouse/mytestdb.db..., PartitionFilters: [isnotnull(to_date_year_month#515), (to_date_year_month#515 = 202710)], Push

In [7]:
# partition by various methods
from datetime import datetime, timedelta
import random
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType

df_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("from_dt", DateType(), True),
    StructField("to_dt", DateType(), True),
])

rows = 100
lst = []
for c in range(rows):
    from_dt = datetime.now().date() + timedelta(days=random.randint(0, 1000))
    to_dt = from_dt + timedelta(days=random.randint(1, 100))

    lst.append([
        c,
        'name-' + str(c).zfill(8),
        None if random.random() < 0.1 else random.randint(1, 100),
        from_dt,
        to_dt
         ])

print(f"lst size: {len(lst)}")

df = spark.createDataFrame(lst, df_schema)

    
spark.sql("create database if not exists testdb")
spark.sql("Show databases").show()
spark.sql("use testdb")
spark.sql("show tables").show()

df.show(truncate = False)

#df.groupBy("to_dt").agg("*", "count").orderBy("to_dt")

# partition by all occuring to_dt values
#df.writeTo("my_partitioned_table").partitionedBy(partitioning.months("to_dt")).createOrReplace() # spark 4.0.0 (buggy)
df.write.partitionBy("to_dt").mode("overwrite").parquet('mydf_1.parquet')  # spark 3.5

# partition by a given number of partitions using HASH
# processing the data must be done as one, only splitting the file to chunks is achieved
df1 = df.repartition(4, "to_dt")
df1.write.mode("overwrite").parquet('mydf_2.parquet')  # spark 3.5

# partition by a given number of partitions using RANGE by sampling, to automatically estimate the ranges
# processing the data must be done as one, only splitting the file to chunks is achieved
df1 = df.repartitionByRange(4, "to_dt")
df1.write.mode("overwrite").parquet('mydf_3.parquet')  # spark 3.5

# limit the records, thus partitioning data in file roughly(!) equally.  full data processing is necessary
df.write.mode("overwrite").option("maxRecordsPerFile", 20).csv("mydf_4.csv")

print(datetime.now().strftime("%Y%m%d %H:%M:%S"), ": Done.")

lst size: 100
+---------+
|namespace|
+---------+
|  default|
| mytestdb|
|   testdb|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+---+-------------+----+----------+----------+
|id |name         |age |from_dt   |to_dt     |
+---+-------------+----+----------+----------+
|0  |name-00000000|72  |2026-06-22|2026-07-20|
|1  |name-00000001|19  |2026-06-27|2026-08-19|
|2  |name-00000002|98  |2026-11-16|2026-12-11|
|3  |name-00000003|52  |2025-09-12|2025-09-13|
|4  |name-00000004|65  |2027-11-30|2028-02-09|
|5  |name-00000005|18  |2027-02-20|2027-03-01|
|6  |name-00000006|59  |2025-11-16|2025-12-15|
|7  |name-00000007|67  |2025-05-05|2025-08-13|
|8  |name-00000008|72  |2025-04-10|2025-04-11|
|9  |name-00000009|29  |2025-04-23|2025-07-26|
|10 |name-00000010|72  |2026-05-31|2026-07-30|
|11 |name-00000011|52  |2026-01-03|2026-02-22|
|12 |name-00000012|46  |2027-12-13|2027-12-17|
|13 |name-00

20250408 15:00:11 : Done.
